# Week 16 — RNNs/GRUs/LSTMs & Sequence Modeling

*Last updated:* 2025-09-09

## Objectives
- [ ] Understand rnns/grus/lstms & sequence modeling
- [ ] Complete guided exercises (theory → code → evaluation)
- [ ] Apply learning in a small project or lab
- [ ] Reflect using self-assessment checklist

## Mini-Theory (Deep Dive)
- Vanishing/exploding gradients; gating; teacher forcing
- Character-level modeling toy task
- When to still use RNNs vs Transformers

## Guided Exercises
    The following exercises are structured to help you learn by doing. Each has **starter code**, **hints**, and **checks**.

In [ ]:
# Exercise: Minimal PyTorch training loop w/ AMP & gradient clipping
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

X = torch.randn(2048, 32)
y = torch.randint(0, 4, (2048,))

class MLP(nn.Module):
    def __init__(self, in_dim=32, hidden=128, out_dim=4):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_dim, hidden), nn.ReLU(), nn.Linear(hidden, out_dim))
    def forward(self, x): return self.net(x)

model = MLP()
opt = torch.optim.AdamW(model.parameters(), lr=3e-4)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

ds = TensorDataset(X, y)
dl = DataLoader(ds, batch_size=64, shuffle=True)

for epoch in range(5):
    for xb, yb in dl:
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(xb)
            loss = F.cross_entropy(logits, yb)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True)
    print("epoch", epoch, "loss", float(loss))
# TODO: add validation loop and accuracy metric

## Project Work
- This week connects to: `code/dl/train_loop.py`
- Implement the **Build** task described in the project README. Tie your notebook experiments into that code (e.g., import your module or save artifacts for the project).

### Deliverable
- A short write-up (5–10 bullets) on **what worked, what didn’t, and what you’ll try next**.

## Self-Assessment Checklist
- [ ] I can explain the key concepts of **RNNs/GRUs/LSTMs & Sequence Modeling** in my own words.
- [ ] I completed the guided exercises and validated outputs.
- [ ] I produced a small artifact (code, plot, or report) and linked it to the project.
- [ ] I captured 3–5 learnings and 2 next steps.

---
**Tip:** Keep each week to ~10 hours: ~3h study, ~3h coding, ~3h project, ~1h reflection.